# Exercise - Building an AI Agent with Tools

In this exercise, you'll build an AI agent that can use tools to enhance its capabilities. You'll learn how to create an agent that can understand when to use tools, process their results, and maintain a coherent conversation.

## Challenge

Imagine you're building a smart coding assistant that needs to:
- Answer programming questions
- Execute code snippets
- Look up documentation
- Perform calculations
- Search through codebases

Instead of hard-coding when to use each capability, your agent should intelligently decide when and how to use its available tools.

## Setup
First, let's import the necessary libraries:

In [1]:
from typing import List, Dict, Any
from dotenv import load_dotenv
from copy import deepcopy
import json

from lib.messages import UserMessage, SystemMessage, ToolMessage
from lib.tooling import tool
from lib.llm import LLM

## Understanding the Components

Before we build our agent, let's understand the key components we'll be working with:

- `LLM`: The language model wrapper that handles tool execution
- `SystemMessage`: Defines the agent's role and behavior
- `UserMessage`: Represents user inputs
- `ToolMessage`: Contains tool execution results
- `tool`: Decorator for creating tools

## Building the Agent Class

Your task is to create an Agent class that can:
1. Initialize with a specific role and set of tools
2. Process user messages
3. Decide when to use tools
4. Handle tool responses

In [2]:
class Agent:
    """An AI Agent that can use tools to help answer questions"""
    
    def __init__(
        self,
        role: str = "Personal Assistant",
        instructions: str = "Help users with any question",
        model: str = "gpt-4o-mini",
        temperature: float = 0.0,
        tools: List[Any] = None
    ):
        """Initialize the agent with its configuration and tools"""
        # TODO 1: Initialize the agent
        # Hint: 
        # - Load environment variables with load_dotenv()
        # - Store agent settings (role, instructions, etc.)
        # - Create an LLM instance with the provided tools
        load_dotenv()
        api_key = os.getenv("OPENAI_API_KEY")
        self.model = model
        self.role = role
        self.instructions = instructions
        self.tools = tools
        self.llm = LLM(model=model, temperature=temperature, tools=tools, api_key=api_key)


    def invoke(self, user_message: str) -> str:
        """Process a user message and return a response"""
        # TODO 2: Set up the conversation
        # Hint:
        # - Create messages list with SystemMessage (role + instructions)
        # - Add UserMessage with user_message
        # - Get initial AI response using self.llm.invoke()

        messages = [
            SystemMessage(
                content=(
                    f"You're an AI Agent and your role is {self.role}. "  
                    f"Your instructions: {self.instructions}"
                )
            )
        ]

        messages.append(UserMessage(content=user_message))

        ai_message = self.llm.invoke(messages)
        messages.append(ai_message)

        # TODO 3: Handle tool calls if needed
        # Hint:
        # - Check if ai_message.tool_calls exists
        # - For each tool call:
        #   * Get function name and arguments
        #   * Execute tool and get result
        #   * Add result as ToolMessage
        # - Get final AI response

        while ai_message.tool_calls:
            for tool_call in ai_message.tool_calls:
                function_args = json.loads(tool_call.function.arguments)
                function_name = tool_call.function.name
                tool_call_id = tool_call.id

                # Find the matching tool
                # tool = next(t for t in self.tools if t.name == function_name else None)
                tool = None
                for t in self.tools:
                    if t.name == function_name:
                        tool = t
                        break
                
                print(tool)
                
                if tool:
                    tool_result = tool(**function_args)
                    messages.append(
                        ToolMessage(
                            content=json.dumps(tool_result),
                            tool_call_id=tool_call_id,
                            name=function_name
                        )
                    )
            
            ai_message = self.llm.invoke(messages)
            messages.append(ai_message)
        
        for m in messages:
            print(m)
        
        return ai_message.content

## Testing Your Agent

Once you've implemented the Agent class, test it with different scenarios:

1. Basic conversation without tools

In [3]:
agent = Agent(role="Coding Assistant")
response = agent.invoke("What is Python? Be concise")
print(response)

content="You're an AI Agent and your role is Coding Assistant. Your instructions: Help users with any question" role='system'
content='What is Python? Be concise' role='user'
content='Python is a high-level, interpreted programming language known for its readability and simplicity. It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. Python is widely used for web development, data analysis, artificial intelligence, scientific computing, and automation, among other applications.' role='assistant' tool_calls=None
Python is a high-level, interpreted programming language known for its readability and simplicity. It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. Python is widely used for web development, data analysis, artificial intelligence, scientific computing, and automation, among other applications.


2. Create a calculator tool

In [4]:
@tool
def calculate(expression: str) -> float:
    """Evaluate a mathematical expression"""
    return eval(expression)

In [5]:
# Create an agent with the calculator tool
math_agent = Agent(
    role="Math Assistant",
    tools=[calculate]
)

In [6]:
response = math_agent.invoke("What is 23 * 45?")
print(response)

<Tool name=calculate params=['expression']>
content="You're an AI Agent and your role is Math Assistant. Your instructions: Help users with any question" role='system'
content='What is 23 * 45?' role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageToolCall(id='call_snhtnNH5f6oLQaswbz3Rp2Us', function=Function(arguments='{"expression":"23 * 45"}', name='calculate'), type='function')]
content='1035' role='tool' tool_call_id='call_snhtnNH5f6oLQaswbz3Rp2Us' name='calculate'
content='The result of \\( 23 \\times 45 \\) is 1035.' role='assistant' tool_calls=None
The result of \( 23 \times 45 \) is 1035.


In [7]:
# Test multiple tool usage
response = math_agent.invoke("If I multiply 3 by 5, what do I get? Then later add 7")
print(response)

<Tool name=calculate params=['expression']>
<Tool name=calculate params=['expression']>
content="You're an AI Agent and your role is Math Assistant. Your instructions: Help users with any question" role='system'
content='If I multiply 3 by 5, what do I get? Then later add 7' role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageToolCall(id='call_dOGZ0opqhTteurYRQj3IS1PX', function=Function(arguments='{"expression": "3 * 5"}', name='calculate'), type='function'), ChatCompletionMessageToolCall(id='call_y43gjRQIiDBbT9PbNFTe0e1w', function=Function(arguments='{"expression": "(3 * 5) + 7"}', name='calculate'), type='function')]
content='15' role='tool' tool_call_id='call_dOGZ0opqhTteurYRQj3IS1PX' name='calculate'
content='22' role='tool' tool_call_id='call_y43gjRQIiDBbT9PbNFTe0e1w' name='calculate'
content='If you multiply 3 by 5, you get 15. If you then add 7 to that result, you get 22.' role='assistant' tool_calls=None
If you multiply 3 by 5, you get 15. If you then a

3. Create a data analyst

In [8]:
GAMES = [
    {"Game": "The Legend of Zelda: Breath of the Wild", "Platform": "Switch", "Score": 98},
    {"Game": "Super Mario Odyssey", "Platform": "Switch", "Score": 97},
    {"Game": "Metroid Prime", "Platform": "GameCube", "Score": 97},
    {"Game": "Super Smash Bros. Brawl", "Platform": "Wii", "Score": 93},
    {"Game": "Mario Kart 8 Deluxe", "Platform": "Switch", "Score": 92},
    {"Game": "Fire Emblem: Awakening", "Platform": "3DS", "Score": 92},
    {"Game": "Donkey Kong Country Returns", "Platform": "Wii", "Score": 87},
    {"Game": "Luigi's Mansion 3", "Platform": "Switch", "Score": 86},
    {"Game": "Pikmin 3", "Platform": "Wii U", "Score": 85},
    {"Game": "Animal Crossing: New Leaf", "Platform": "3DS", "Score": 88},
]

In [9]:
@tool
def get_games(num_games:int=1, top:bool=True) -> str:
    """
    Returns the top or bottom N games with highest or lowest scores.    
    args:
        num_games (int): Number of games to return (default is 1)
        top (bool): If True, return top games, otherwise return bottom (default is True)
    """
    # Sort the games list by Score
    # If top is True, descending order
    sorted_games = sorted(GAMES, key=lambda x: x['Score'], reverse=top)
    
    # Return the N games
    return sorted_games[:num_games]

@tool
def average_score() -> float:
    """Return the average score across all games."""
    return sum(g["Score"] for g in GAMES) / len(GAMES)

@tool
def platform_mean(platform: str) -> float:
    """Return the mean score for a given platform (case-insensitive)."""
    p = platform.lower()
    subset = [g["Score"] for g in GAMES if g["Platform"].lower() == p]
    if not subset:
        return float("nan")
    return sum(subset) / len(subset)

@tool
def count_games(platform: str | None = None) -> int:
    """Count games overall or for a specific platform."""
    if platform is None:
        return len(GAMES)
    return sum(1 for g in GAMES if g["Platform"].lower() == platform.lower())

@tool
def platform_distribution() -> dict:
    """
    Returns the count of games per platform.
    """
    counts = {}
    for g in GAMES:
        counts[g["Platform"]] = counts.get(g["Platform"], 0) + 1
    return counts

In [10]:
# Create an agent with the multiple tools
data_analyst_agent = Agent(
    role="Game Stats Assistant",
    instructions="You can bring insights about a game dataset based on users questions",
    tools=[get_games]
)

In [11]:
response = data_analyst_agent.invoke("What's the best game in the dataset?")
print(response)

<Tool name=get_games params=['num_games', 'top']>
content="You're an AI Agent and your role is Game Stats Assistant. Your instructions: You can bring insights about a game dataset based on users questions" role='system'
content="What's the best game in the dataset?" role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageToolCall(id='call_9nar7VW4zJ836naGM4Y0UW6o', function=Function(arguments='{"num_games":1,"top":true}', name='get_games'), type='function')]
content='[{"Game": "The Legend of Zelda: Breath of the Wild", "Platform": "Switch", "Score": 98}]' role='tool' tool_call_id='call_9nar7VW4zJ836naGM4Y0UW6o' name='get_games'
content='The best game in the dataset is **The Legend of Zelda: Breath of the Wild** for the Switch, with a score of **98**.' role='assistant' tool_calls=None
The best game in the dataset is **The Legend of Zelda: Breath of the Wild** for the Switch, with a score of **98**.


In [12]:
response = data_analyst_agent.invoke("What's the worst game in the dataset?")
print(response)

<Tool name=get_games params=['num_games', 'top']>
content="You're an AI Agent and your role is Game Stats Assistant. Your instructions: You can bring insights about a game dataset based on users questions" role='system'
content="What's the worst game in the dataset?" role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageToolCall(id='call_xmoif45TuhMk3uf3LFNcNtV8', function=Function(arguments='{"num_games":1,"top":false}', name='get_games'), type='function')]
content='[{"Game": "Pikmin 3", "Platform": "Wii U", "Score": 85}]' role='tool' tool_call_id='call_xmoif45TuhMk3uf3LFNcNtV8' name='get_games'
content='The worst game in the dataset is "Pikmin 3" for the Wii U, with a score of 85.' role='assistant' tool_calls=None
The worst game in the dataset is "Pikmin 3" for the Wii U, with a score of 85.


In [13]:
game_stats_agent = Agent(
    role="Game Stats Assistant",
    instructions="Answer questions about the provided Nintendo games dataset. Use tools when helpful.",
    tools=[get_games, average_score, platform_mean, count_games, platform_distribution],
)

In [14]:
print(game_stats_agent.invoke("What is the average score?"))

<Tool name=average_score params=[]>
content="You're an AI Agent and your role is Game Stats Assistant. Your instructions: Answer questions about the provided Nintendo games dataset. Use tools when helpful." role='system'
content='What is the average score?' role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageToolCall(id='call_LeqMXriYeRo00QPaqyqzV7Wr', function=Function(arguments='{}', name='average_score'), type='function')]
content='91.5' role='tool' tool_call_id='call_LeqMXriYeRo00QPaqyqzV7Wr' name='average_score'
content='The average score across all games is 91.5.' role='assistant' tool_calls=None
The average score across all games is 91.5.


In [15]:
print(game_stats_agent.invoke("Show me the top 3 games by score."))

<Tool name=get_games params=['num_games', 'top']>
content="You're an AI Agent and your role is Game Stats Assistant. Your instructions: Answer questions about the provided Nintendo games dataset. Use tools when helpful." role='system'
content='Show me the top 3 games by score.' role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageToolCall(id='call_BRtkbiNmsVZKtOVXRJ79Koxx', function=Function(arguments='{"num_games":3,"top":true}', name='get_games'), type='function')]
content='[{"Game": "The Legend of Zelda: Breath of the Wild", "Platform": "Switch", "Score": 98}, {"Game": "Super Mario Odyssey", "Platform": "Switch", "Score": 97}, {"Game": "Metroid Prime", "Platform": "GameCube", "Score": 97}]' role='tool' tool_call_id='call_BRtkbiNmsVZKtOVXRJ79Koxx' name='get_games'
content='The top 3 games by score are:\n\n1. **The Legend of Zelda: Breath of the Wild** - Score: 98 (Platform: Switch)\n2. **Super Mario Odyssey** - Score: 97 (Platform: Switch)\n3. **Metroid Prime** 

In [16]:
print(game_stats_agent.invoke("How many Switch games are there?"))

<Tool name=count_games params=['platform']>
content="You're an AI Agent and your role is Game Stats Assistant. Your instructions: Answer questions about the provided Nintendo games dataset. Use tools when helpful." role='system'
content='How many Switch games are there?' role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageToolCall(id='call_E0fPMr4KFiWeXc6D77Hu0tij', function=Function(arguments='{"platform":"Switch"}', name='count_games'), type='function')]
content='4' role='tool' tool_call_id='call_E0fPMr4KFiWeXc6D77Hu0tij' name='count_games'
content='There are 4 Nintendo Switch games in the dataset.' role='assistant' tool_calls=None
There are 4 Nintendo Switch games in the dataset.


In [17]:
print(game_stats_agent.invoke("What's the average score on 3DS?"))

<Tool name=platform_mean params=['platform']>
content="You're an AI Agent and your role is Game Stats Assistant. Your instructions: Answer questions about the provided Nintendo games dataset. Use tools when helpful." role='system'
content="What's the average score on 3DS?" role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageToolCall(id='call_d2SUNjlpoXh4nUc4pxKUnZ95', function=Function(arguments='{"platform":"3DS"}', name='platform_mean'), type='function')]
content='90.0' role='tool' tool_call_id='call_d2SUNjlpoXh4nUc4pxKUnZ95' name='platform_mean'
content='The average score on the 3DS is 90.0.' role='assistant' tool_calls=None
The average score on the 3DS is 90.0.


In [18]:
print(game_stats_agent.invoke("What is the distribution of platforms?"))

<Tool name=platform_distribution params=[]>
content="You're an AI Agent and your role is Game Stats Assistant. Your instructions: Answer questions about the provided Nintendo games dataset. Use tools when helpful." role='system'
content='What is the distribution of platforms?' role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageToolCall(id='call_WtnK5sZYsOaVIk3S99uXLo3Z', function=Function(arguments='{}', name='platform_distribution'), type='function')]
content='{"Switch": 4, "GameCube": 1, "Wii": 2, "3DS": 2, "Wii U": 1}' role='tool' tool_call_id='call_WtnK5sZYsOaVIk3S99uXLo3Z' name='platform_distribution'
content='The distribution of platforms in the dataset is as follows:\n\n- Switch: 4 games\n- GameCube: 1 game\n- Wii: 2 games\n- 3DS: 2 games\n- Wii U: 1 game' role='assistant' tool_calls=None
The distribution of platforms in the dataset is as follows:

- Switch: 4 games
- GameCube: 1 game
- Wii: 2 games
- 3DS: 2 games
- Wii U: 1 game
